#### Canadian UAVs Coding Challenge
Ahmed Ibrahim Saeed; ahmed.saeed@ucalgary.ca

Created in Jupyter Notebook for a concise approach towards solving the problem and testing the code output.

In [6]:
import csv
import json
import math
import argparse

EARTH_RADIUS_M = 6371000.0
def haversine_m(lat1, lon1, lat2, lon2): # This is the great circle distance between two WGS84 lat/lon points in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = (math.sin(dphi / 2) ** 2
         + math.cos(phi1) * math.cos(phi2) * (math.sin(dlambda / 2) ** 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return EARTH_RADIUS_M * c

def is_valid_wgs84(lat, lon):
    return (-90.0 <= lat <= 90.0) and (-180.0 <= lon <= 180.0)

def load_sensor1_csv(path): # Expected/required columns: id,latitude,longitude
    out = []
    with open(path, "r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for r in reader:
            lat = float(r["latitude"])
            lon = float(r["longitude"])
            if not is_valid_wgs84(lat, lon):
                continue
            out.append({
                "id": str(r["id"]), # JSON keys must be strings
                "lat": lat,
                "lon": lon
            })
    return out

def load_sensor2_json(path): # Expected/required format: list of {id, latitude, longitude}
    data = json.load(open(path, "r", encoding="utf-8"))
    if not isinstance(data, list):
        raise ValueError("Sensor2 JSON must be a list of objects.")
    out = []
    for item in data:
        lat = float(item["latitude"])
        lon = float(item["longitude"])
        if not is_valid_wgs84(lat, lon):
            continue
        out.append({
            "id": int(item["id"]),
            "lat": lat,
            "lon": lon
        })
    return out

def correlate_greedy_one_to_one(sensor1, sensor2, threshold_m=200.0): # Building all candidate pairs within threshold
    candidates = []
    for a in sensor1:
        for b in sensor2:
            d = haversine_m(a["lat"], a["lon"], b["lat"], b["lon"])
            if d <= threshold_m:
                candidates.append((d, a["id"], b["id"]))
    candidates.sort(key=lambda x: x[0]) # Greedy minimum distance matching
    used1 = set()
    used2 = set()
    matched = {}
    for d, id1, id2 in candidates:
        if id1 in used1 or id2 in used2:
            continue
        matched[id1] = id2
        used1.add(id1)
        used2.add(id2)
    return matched

def main():
    csv_path = "SensorData1.csv"
    json_path = "SensorData2.json"
    out_path = "Output.json"
    threshold_m = 200.0
    s1 = load_sensor1_csv(csv_path)
    s2 = load_sensor2_json(json_path)
    result = correlate_greedy_one_to_one(s1, s2, threshold_m)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(result, f, indent=4)
    print(f"Matched pairs: {len(result)}")
    print(f"Wrote output to: {out_path}")

main()

Matched pairs: 11
Wrote output to: Output.json


#### JSON Output Readings

In [7]:
import json
output_path = "Output.json"

with open(output_path, "r", encoding="utf-8") as f:
    results = json.load(f)
print(f"Total matched pairs: {len(results)}\n")

for sensor1_id, sensor2_id in results.items():
    print(f"Sensor 1 ID {sensor1_id}  -->  Sensor 2 ID {sensor2_id}")

Total matched pairs: 11

Sensor 1 ID 19  -->  Sensor 2 ID 13
Sensor 1 ID 11  -->  Sensor 2 ID 37
Sensor 1 ID 16  -->  Sensor 2 ID 33
Sensor 1 ID 12  -->  Sensor 2 ID 17
Sensor 1 ID 23  -->  Sensor 2 ID 40
Sensor 1 ID 15  -->  Sensor 2 ID 23
Sensor 1 ID 6  -->  Sensor 2 ID 19
Sensor 1 ID 2  -->  Sensor 2 ID 9
Sensor 1 ID 8  -->  Sensor 2 ID 12
Sensor 1 ID 21  -->  Sensor 2 ID 29
Sensor 1 ID 17  -->  Sensor 2 ID 5
